# Project III: A MongoDB & API EDA project

## You recently created a new company in the GAMING industry. The company will have the following scheme:

- 20 Designers
- 5 UI/UX Engineers
- 10 Frontend Developers
- 15 Data Engineers
- 5 Backend Developers
- 20 Account Managers
- 1 Maintenance guy that loves basketball
- 10 Executives
- 1 CEO/President.

As a data engineer you have asked all the employees to show their preferences on where to place the new office. Your goal is to place the new company offices in the best place for the company to grow. You have to find a place that more or less covers all the following requirements (note that it's impossible to cover all requirements, so you have to prioritize at your glance):

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company staff have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Account managers need to travel a lot.
- Everyone in the company is between 25 and 40, give them some place to go party.
- The CEO is vegan.
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.


![mapa](https://res.cloudinary.com/sblink/_/og_image.png)






### Connecting to MongoDB and importing libraries

In [1]:
import json
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from pymongo import MongoClient
import pandas as pd
import time 


# importing the mongo.py file/library
import mongo as mongos
import api as api


In [2]:
# connecting
c=mongos.connecting()


In [3]:
# calling the mongo extraction function
world_offices_data= mongos.mongo_search(c)

In [4]:
# calling the cleaning and transformation function

df_sorted=mongos.cleaning_mongo(world_offices_data)
df_sorted

# These are the 5 offices we will be working on

,company_name,total_money_raised,lat,lon,state_code,country_code,city,schools,airports,train stations,clubs,vegan places,basketball places,pet grooming
11,Carrier IQ,$42M,37.395578,-122.083329,CA,USA,Mountain View,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Digg,$45M,37.764726,-122.394523,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99designs,$35M,37.795531,-122.400598,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Minted,$52.7M,37.797435,-122.403175,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Netbiscuits,$27M,37.755037,-122.426442,CA,USA,San Francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN


After pushing, view your own notebook on [nbviewer](https://nbviewer.org/)

In [5]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [6]:

usa_lat = 37.7576171
usa_lon = -122.5776844
usa_map = Map(location = [usa_lat, usa_lon], zoom_start = 3)

for index, row in world_offices_data.iterrows():
    all_together = {"location": [row["lat"], row["lon"]]}
    icon = Icon (
                icon = "building",
                prefix = "fa",
                color = "ce40c2",
                icon_color = "white")        
       
    # 2. With the icon: I pass that to the Marker

    new_marker = Marker(**all_together, icon = icon)
    
    # 3. Add a Marker per row
    
    new_marker.add_to(usa_map)

usa_map

/var/folders/tb/spjhzdz13378nkrzqyjljkjm0000gn/T/ipykernel_10451/1006630026.py:7: UserWarning: color argument of Icon should be one of: {'lightgray', 'lightgreen', 'darkblue', 'red', 'lightred', 'gray', 'darkpurple', 'white', 'cadetblue', 'lightblue', 'purple', 'beige', 'blue', 'pink', 'orange', 'darkgreen', 'black', 'green', 'darkred'}.
  icon = Icon (


### As we can see most of the startups that meet the following requirements are in the west coast:

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

- Developers like to be near successful tech startups that have raised at least 1 Million dollars.

### We will focus on Mountain View and San Francisco

## Let's start with the API

# Just to remember, these are the parameters we will be evaluating:
- preschool_c = "12056"
- airport_c = "19040"
- train_station_c = "19046"
- nightlife_spot_c = "10032"
- vegetarian_vegan_c = "13377"
- basketball_stadium_c = "18008"
- pet_grooming_c = "11062"


In [7]:
from getpass import getpass
from dotenv import load_dotenv
import os
import requests

## Let's start with schools

#### San Francisco and Mountain View


In [9]:
dfs=api.categories(df_sorted)

NameError: name 'row' is not defined

In [ ]:
def factor_from_foursquare(one_element):

    lat = row['lat']
    lon = row['lon']
    
    name = one_element["name"]
    distance = one_element["distance"]
    address = one_element["location"]["formatted_address"]
    lat = one_element["geocodes"]["main"]["latitude"]
    lon = one_element["geocodes"]["main"]["longitude"]
    category = one_element["categories"][0]["name"]
    
    dict_ = {"name": name, "distance": distance, "lat": lat, "lon": lon, "category":category}
    
    return dict_
    
def categories():
    
    dfs = []

    categories = ["12056", "19040", "19046", "10032", "13377", "18008", "11134"]

    for category in categories:

        category_df = pd.DataFrame(columns=["name", "distance", "lat", "lon", "category"])

        for index, row in df_sorted.iterrows():
            lat = row['lat']
            lon = row['lon']
            name = row['company_name']
            city = row['city']

            response = query_from_foursquare("", lat, lon, 1000, category)

            new_list = []
            for i in response["results"]:
                factor_dict = factor_from_foursquare(i)
                factor_dict['company_name'] = name
                factor_dict['city'] = city
                new_list.append(factor_dict)



            # append results to category dataframe


            category_df = category_df.append(new_list, ignore_index=True)

        dfs.append(category_df)
        return dfs


In [ ]:

dfs[4]['category'] = 'vegan'


In [ ]:
dfs[1] = dfs[1].assign(company_name="", city="")

In [ ]:

# Here the idea is to count the number of locations for each category and for each office and add it to our main
# dataframe, which is df_sorted. As you can see I do "df_sorted.columns[7:]" to start iterating on the 7th column,
# because that's where I will be starting adding the count.

# Iterate through columns of df_sorted
for i, col in enumerate(df_sorted.columns[7:]):

    # Count the number of rows in dfs[i] for each company name in df_sorted
    counts = []
    for index, row in df_sorted.iterrows():
        company_name = row["company_name"]
        count = dfs[i][dfs[i]["company_name"] == company_name].shape[0]
        counts.append(count)

    # Update the corresponding column in df_sorted with the counts
    df_sorted.iloc[:, i+7] = counts


In [ ]:

usa_lat = 37.7653307
usa_lon = -122.419473616
usa_map = Map(location = [usa_lat, usa_lon], zoom_start = 13)
for df in dfs:
    for index, row in df.iterrows():

        all_together = {"location": [row["lat"], row["lon"]]}
    
        if row["category"] == "Child Care Service" or row["category"] == "Preschool":
            icon = Icon (
                icon = "graduation-cap",
                prefix = "fa",
                color = "blue",
                icon_color = "white")
        
        elif row["category"] == "Metro Station":
            icon = Icon (
                icon = "train-tram",
                prefix = "fa",
                color = "beige",
                icon_color = "black")
        
        elif row["category"] == "Night Club":
            icon = Icon (
                icon = "champagne-glasses",
                prefix = "fa",
                color = "lightblue",
                icon_color = "black") 
    
        elif row["category"] == "vegan":
            icon = Icon (
                icon = "leaf",
                prefix = "fa",
                color = "yellow",
                icon_color = "black")
        
    
        elif row["category"] == "Basketball Court":
            icon = Icon (
                icon = "basketball",
                prefix = "fa",
                color = "green",
                icon_color = "white")
        
        elif row["category"] == "Pet Grooming Service":
            icon = Icon (
                icon = "dog",
                prefix = "fa",
                color = "orange",
                icon_color = "white")
    
        
       
    # 2. With the icon: I pass that to the Marker

        new_marker = Marker(**all_together, icon = icon)
    
    # 3. Add a Marker per row
    
        new_marker.add_to(usa_map)

usa_map